In [ ]:
# ==========================================================
#  Importando as bibliotecas necessárias
# ==========================================================

import requests  # Biblioteca para fazer requisições HTTP, vamos usar para chamar a API de países
from pyspark.sql import SparkSession  # Para criar e manipular DataFrames no Spark

# Criando a sessão Spark
spark = SparkSession.builder \
    .appName("API REST Countries") \
    .getOrCreate()

print("🚀 SparkSession criada com sucesso!")


# ==========================================================
# Definindo a URL da API REST Countries
# ==========================================================

url = "https://restcountries.com/v3.1/all"  # Endpoint que retorna informações de todos os países
print(f"🔗 URL da API: {url}")

# ==========================================================
# Fazendo a requisição para a API
# ==========================================================

response = requests.get(url)  # Enviando uma requisição GET para a API

# ==========================================================
# Verificando se a requisição foi bem-sucedida
# ==========================================================

if response.status_code == 200:
    print("✅ Requisição feita com sucesso!")
    
    # ==========================================================
    # Convertendo a resposta da API para formato JSON
    # ==========================================================

    countries_data = response.json()  # A resposta vem como texto, então transformamos para um objeto Python (lista de dicionários)

    print(f"📊 Total de países retornados: {len(countries_data)}")

    # ==========================================================
    # Preparando os dados para criar um DataFrame Spark
    # ==========================================================

    # Vamos criar uma lista vazia para armazenar as informações que queremos
    countries_list = []

    # Vamos percorrer cada país na resposta da API
    for country in countries_data:
        # Extraindo o nome comum do país
        nome_resumido = country.get('cca2', 'Não informado')

        # Extraindo o nome comum do país
        name = country.get("name", {}).get("common")

        # Extraindo o nome oficial do país
        official_name = country.get("name", {}).get("official")

        # Extraindo a capital, que pode ser uma lista (ou não existir)
        capital = country.get("capital", ["Não informado"])[0] if country.get("capital") else "Não informado"

        # Extraindo a região do país (ex: Europe, Asia)
        region = country.get("region")

        # Extraindo a sub-região (ex: Western Europe)
        subregion = country.get("subregion", "Não informado")

        # População do país
        population = country.get("population")

        # Área do país em km²
        area = country.get("area")

        # Idiomas falados no país (pode ter vários, então juntamos em uma string)
        languages = ", ".join(country.get("languages", {}).values()) if country.get("languages") else "Não informado"

        # Moedas usadas no país (juntamos o nome das moedas em uma string)
        currencies = ", ".join([v.get("name") for v in country.get("currencies", {}).values()]) if country.get("currencies") else "Não informado"

        # URL da imagem da bandeira
        flag = country.get("flags", {}).get("png", "Não informado")

        # Adicionamos os dados coletados na nossa lista como uma tupla
        countries_list.append((nome_resumido,name, official_name, capital, region, subregion, population, area, languages, currencies, flag))

    # ==========================================================
    #  Criando o DataFrame Spark com os dados coletados
    # ==========================================================

    # Definimos os nomes das colunas para o DataFrame
    df_countries = spark.createDataFrame(
        countries_list,
        ["nome_resumido","nome", "nome_oficial", "capital", "regiao", "subregiao", "populacao", "area_km2", "idiomas", "moedas", "bandeira_url"]
    )

    # ==========================================================
    # Exibindo o DataFrame para visualização
    # ==========================================================

    display(df_countries)  # Comando para visualizar os dados em tabela

    # ==========================================================
    # Criando uma Temp View para consultas SQL
    # ==========================================================

    print("✅ df_countries criado com sucesso!")

else:
    # Caso a requisição falhe, exibimos o status code
    print(f"❌ Erro na requisição. Status code: {response.status_code}")
